In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pybaseball as pyb


pyb.cache.enable()
pd.set_option('display.max_columns',100)
pd.set_option('display.max_rows',100)


In [4]:
fname = 'gl2020_23/gl2023.txt'
df = pd.read_csv(fname, header=None)

In [5]:
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,...,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160
0,20230330,0,Thu,MIL,NL,1,CHN,NL,1,0,4,51,D,NaN,NaN,NaN,CHI11,36054.0,141,000000000,00400000x,29,4,0,0,0,0,0,0,0,5,0,12,0,0,2,0,7,4,4,4,0,0,24,12,1,0,1,0,30,...,adamw002,Willy Adames,6,tellr001,Rowdy Tellez,3,contw002,William Contreras,2,urial001,Luis Urias,5,mitcg001,Garrett Mitchell,8,andeb006,Brian Anderson,9,turab002,Brice Turang,4,hoern001,Nico Hoerner,4,swand001,Dansby Swanson,6,happi001,Ian Happ,7,bellc002,Cody Bellinger,8,manct001,Trey Mancini,10,gomey001,Yan Gomes,2,hosme001,Eric Hosmer,3,wisdp001,Patrick Wisdom,5,mastm001,Miles Mastrobuoni,9,NaN,Y
1,20230330,0,Thu,PIT,NL,1,CIN,NL,1,5,4,54,D,NaN,NaN,NaN,CIN09,44063.0,182,001300010,100120000,30,6,1,0,1,4,1,1,0,9,0,11,2,0,1,0,9,5,4,4,0,0,27,9,1,0,2,0,33,...,mccua001,Andrew McCutchen,10,santc002,Carlos Santana,3,smitc008,Canaan Smith-Njigba,9,hayek001,Ke'Bryan Hayes,5,suwij001,Jack Suwinski,8,bae-j001,Ji Hwan Bae,4,hedga001,Austin Hedges,2,indij001,Jonathan India,4,friet001,TJ Friedl,8,fralj001,Jake Fraley,10,stept001,Tyler Stephenson,2,voslj001,Jason Vosler,3,myerw001,Wil Myers,9,stees001,Spencer Steer,5,bensw001,Will Benson,7,garcj007,Jose Garcia,6,NaN,Y
2,20230330,0,Thu,ARI,NL,1,LAN,NL,1,2,8,51,N,NaN,NaN,NaN,LOS03,52075.0,155,110000000,00203201x,28,4,1,0,0,2,0,1,1,0,0,8,0,0,2,0,1,5,8,8,0,0,24,7,1,0,0,0,34,...,gurrl001,Lourdes Gurriel,7,walkc002,Christian Walker,3,longe001,Evan Longoria,5,ahmen001,Nick Ahmed,6,carrc005,Corbin Carroll,8,moreg001,Gabriel Moreno,2,mccaj003,Jake McCarthy,9,bettm001,Mookie Betts,9,freef001,Freddie Freeman,3,smitw003,Will Smith,2,muncm001,Max Muncy,5,martj006,J.D. Martinez,10,perad001,David Peralta,7,vargm001,Miguel Vargas,4,outmj002,James Outman,8,rojam002,Miguel Rojas,6,NaN,Y
3,20230330,0,Thu,NYN,NL,1,MIA,NL,1,5,3,54,D,NaN,NaN,NaN,MIA02,31397.0,162,001002200,000003000,32,8,1,0,0,5,0,2,1,5,0,5,1,0,0,0,8,4,3,3,0,0,27,8,0,0,2,0,30,...,lindf001,Francisco Lindor,6,alonp001,Pete Alonso,3,mcnej002,Jeff McNeil,4,canhm001,Mark Canha,7,voged001,Daniel Vogelbach,10,escoe001,Eduardo Escobar,5,narvo001,Omar Narvaez,2,arral001,Luis Arraez,4,seguj002,Jean Segura,5,coopg002,Garrett Cooper,3,chisj001,Jazz Chisholm,8,solej001,Jorge Soler,10,garca003,Avisail Garcia,9,delab001,Bryan De La Cruz,7,stalj001,Jacob Stallings,2,wendj002,Joey Wendle,6,NaN,Y
4,20230330,0,Thu,COL,NL,1,SDN,NL,1,7,2,54,N,NaN,NaN,NaN,SAN02,45103.0,176,100031200,100100000,44,17,4,0,3,7,0,0,0,1,0,17,1,0,0,0,11,4,2,2,0,1,27,9,3,0,2,0,32,...,blacc001,Charlie Blackmon,10,cronc002,C.J. Cron,3,monte001,Elehuris Montero,5,mcmar001,Ryan McMahon,4,diaze005,Elias Diaz,2,casth001,Harold Castro,7,tovae001,Ezequiel Tovar,6,grist001,Trent Grisham,8,sotoj001,Juan Soto,7,machm001,Manny Machado,5,bogax001,Xander Bogaerts,6,cronj001,Jake Cronenworth,3,carpm002,Matt Carpenter,10,nolaa002,Austin Nola,2,kim-h002,Ha-Seong Kim,4,dahld001,David Dahl,9,NaN,Y


In [21]:
colnames = ['date','dblheader_code','day_of_week','team_v','league_v','game_no_v',
           'team_h','league_h','game_no_h', 'runs_v', 'runs_h','outs_total','day_night',
            'completion_info','forfeit_info','protest_info','ballpark_id','attendance','game_minutes',
            'linescore_v','linescore_h',
           'AB_v','H_v','2B_v','3B_v','HR_v','RBI_v','SH_v','SF_v','HBP_v','BB_v','IBB_v','SO_v',
            'SB_v', 'CS_v','GIDP_v','CI_v','LOB_v',
            'P_num_v','ERind_v','ERteam_v','WP_v','balk_v',
            'PO_v','ASST_v','ERR_v','PB_v','DP_v','TP_v',
           'AB_h', 'H_h', '2B_h', '3B_h', 'HR_h', 'RBI_h', 'SH_h', 'SF_h', 'HBP_h', 'BB_h', 'IBB_h','SO_h',
            'SB_h', 'CS_h', 'GIDP_h', 'CI_h', 'LOB_h',
            'P_num_h', 'ERind_h', 'ERteam_h', 'WP_h', 'balk_h',
            'PO_h', 'ASST_h', 'ERR_h', 'PB_h', 'DP_h', 'TP_h',
            'ump_HB_id', 'ump_HB_name','ump_1B_id', 'ump_1B_name','ump_2B_id', 'ump_2B_name',
            'ump_3B_id', 'ump_3B_name','ump_LF_id', 'ump_LF_name','ump_RF_id', 'ump_RF_name',
            'mgr_id_v', 'mgr_name_v', 'mgr_id_h', 'mgr_name_h',
            'pitcher_id_w','pitcher_name_w','pitcher_id_l','pitcher_name_l','pitcher_id_s','pitcher_name_s',
            'GWRBI_id','GWRBI_name','pitcher_start_id_v','pitcher_start_name_v','pitcher_start_id_h','pitcher_start_name_h',
            'batter1_name_v', 'batter1_id_v', 'batter1_pos_v', 'batter2_name_v', 'batter2_id_v', 'batter2_pos_v',
            'batter3_name_v', 'batter3_id_v', 'batter3_pos_v', 'batter4_name_v', 'batter4_id_v', 'batter4_pos_v',
            'batter5_name_v', 'batter5_id_v', 'batter5_pos_v', 'batter6_name_v', 'batter6_id_v', 'batter6_pos_v',
            'batter7_name_v', 'batter7_id_v', 'batter7_pos_v', 'batter8_name_v', 'batter8_id_v', 'batter8_pos_v',
            'batter9_name_v', 'batter9_id_v', 'batter9_pos_v', 'batter1_name_h', 'batter1_id_h', 'batter1_pos_h',
            'batter2_name_h', 'batter2_id_h', 'batter2_pos_h', 'batter3_name_h', 'batter3_id_h', 'batter3_pos_h',
            'batter4_name_h', 'batter4_id_h', 'batter4_pos_h', 'batter5_name_h', 'batter5_id_h', 'batter5_pos_h',
            'batter6_name_h', 'batter6_id_h', 'batter6_pos_h', 'batter7_name_h', 'batter7_id_h', 'batter7_pos_h',
            'batter8_name_h', 'batter8_id_h', 'batter8_pos_h', 'batter9_name_h', 'batter9_id_h', 'batter9_pos_h',           
           'misc_info','acqui_info'
           ]

df.columns = colnames
df

,date,dblheader_code,day_of_week,team_v,league_v,game_no_v,team_h,league_h,game_no_h,runs_v,runs_h,outs_total,day_night,completion_info,forfeit_info,protest_info,ballpark_id,attendance,game_minutes,linescore_v,linescore_h,AB_v,H_v,2B_v,3B_v,HR_v,RBI_v,SH_v,SF_v,HBP_v,BB_v,IBB_v,SO_v,SB_v,CS_v,GIDP_v,CI_v,LOB_v,P_num_v,ERind_v,ERteam_v,WP_v,balk_v,PO_v,ASST_v,ERR_v,PB_v,DP_v,TP_v,AB_h,...,batter3_name_v,batter3_id_v,batter3_pos_v,batter4_name_v,batter4_id_v,batter4_pos_v,batter5_name_v,batter5_id_v,batter5_pos_v,batter6_name_v,batter6_id_v,batter6_pos_v,batter7_name_v,batter7_id_v,batter7_pos_v,batter8_name_v,batter8_id_v,batter8_pos_v,batter9_name_v,batter9_id_v,batter9_pos_v,batter1_name_h,batter1_id_h,batter1_pos_h,batter2_name_h,batter2_id_h,batter2_pos_h,batter3_name_h,batter3_id_h,batter3_pos_h,batter4_name_h,batter4_id_h,batter4_pos_h,batter5_name_h,batter5_id_h,batter5_pos_h,batter6_name_h,batter6_id_h,batter6_pos_h,batter7_name_h,batter7_id_h,batter7_pos_h,batter8_name_h,batter8_id_h,batter8_pos_h,batter9_name_h,batter9_id_h,batter9_pos_h,misc_info,acqui_info
0,20230330,0,Thu,MIL,NL,1,CHN,NL,1,0,4,51,D,NaN,NaN,NaN,CHI11,36054.0,141,000000000,00400000x,29,4,0,0,0,0,0,0,0,5,0,12,0,0,2,0,7,4,4,4,0,0,24,12,1,0,1,0,30,...,adamw002,Willy Adames,6,tellr001,Rowdy Tellez,3,contw002,William Contreras,2,urial001,Luis Urias,5,mitcg001,Garrett Mitchell,8,andeb006,Brian Anderson,9,turab002,Brice Turang,4,hoern001,Nico Hoerner,4,swand001,Dansby Swanson,6,happi001,Ian Happ,7,bellc002,Cody Bellinger,8,manct001,Trey Mancini,10,gomey001,Yan Gomes,2,hosme001,Eric Hosmer,3,wisdp001,Patrick Wisdom,5,mastm001,Miles Mastrobuoni,9,NaN,Y
1,20230330,0,Thu,PIT,NL,1,CIN,NL,1,5,4,54,D,NaN,NaN,NaN,CIN09,44063.0,182,001300010,100120000,30,6,1,0,1,4,1,1,0,9,0,11,2,0,1,0,9,5,4,4,0,0,27,9,1,0,2,0,33,...,mccua001,Andrew McCutchen,10,santc002,Carlos Santana,3,smitc008,Canaan Smith-Njigba,9,hayek001,Ke'Bryan Hayes,5,suwij001,Jack Suwinski,8,bae-j001,Ji Hwan Bae,4,hedga001,Austin Hedges,2,indij001,Jonathan India,4,friet001,TJ Friedl,8,fralj001,Jake Fraley,10,stept001,Tyler Stephenson,2,voslj001,Jason Vosler,3,myerw001,Wil Myers,9,stees001,Spencer Steer,5,bensw001,Will Benson,7,garcj007,Jose Garcia,6,NaN,Y
2,20230330,0,Thu,ARI,NL,1,LAN,NL,1,2,8,51,N,NaN,NaN,NaN,LOS03,52075.0,155,110000000,00203201x,28,4,1,0,0,2,0,1,1,0,0,8,0,0,2,0,1,5,8,8,0,0,24,7,1,0,0,0,34,...,gurrl001,Lourdes Gurriel,7,walkc002,Christian Walker,3,longe001,Evan Longoria,5,ahmen001,Nick Ahmed,6,carrc005,Corbin Carroll,8,moreg001,Gabriel Moreno,2,mccaj003,Jake McCarthy,9,bettm001,Mookie Betts,9,freef001,Freddie Freeman,3,smitw003,Will Smith,2,muncm001,Max Muncy,5,martj006,J.D. Martinez,10,perad001,David Peralta,7,vargm001,Miguel Vargas,4,outmj002,James Outman,8,rojam002,Miguel Rojas,6,NaN,Y
3,20230330,0,Thu,NYN,NL,1,MIA,NL,1,5,3,54,D,NaN,NaN,NaN,MIA02,31397.0,162,001002200,000003000,32,8,1,0,0,5,0,2,1,5,0,5,1,0,0,0,8,4,3,3,0,0,27,8,0,0,2,0,30,...,lindf001,Francisco Lindor,6,alonp001,Pete Alonso,3,mcnej002,Jeff McNeil,4,canhm001,Mark Canha,7,voged001,Daniel Vogelbach,10,escoe001,Eduardo Escobar,5,narvo001,Omar Narvaez,2,arral001,Luis Arraez,4,seguj002,Jean Segura,5,coopg002,Garrett Cooper,3,chisj001,Jazz Chisholm,8,solej001,Jorge Soler,10,garca003,Avisail Garcia,9,delab001,Bryan De La Cruz,7,stalj001,Jacob Stallings,2,wendj002,Joey Wendle,6,NaN,Y
4,20230330,0,Thu,COL,NL,1,SDN,NL,1,7,2,54,N,NaN,NaN,NaN,SAN02,45103.0,176,100031200,100100000,44,17,4,0,3,7,0,0,0,1,0,17,1,0,0,0,11,4,2,2,0,1,27,9,3,0,2,0,32,...,blacc001,Charlie Blackmon,10,cronc002,C.J. Cron,3,monte001,Elehuris Montero,5,mcmar001,Ryan McMahon,4,diaze005,Elias Diaz,2,casth001,Harold Castro,7,tovae001,Ezequiel Tovar,6,grist001,Trent Grisham,8,sotoj001,Juan Soto,7,machm001,Manny Machado,5,bogax001,Xander Bogaerts,6,cronj001,Jake Cronenworth,3,carpm002,Matt Carpenter,10,nolaa002,Austin Nola,2,kim-h002,Ha-Seong Kim,4,dahld001,David Dahl,9,NaN,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,.

In [15]:
teams = np.array([df.groupby('team_v').agg('first').index])
teams

array([['ANA', 'ARI', 'ATL', 'BAL', 'BOS', 'CHA', 'CHN', 'CIN', 'CLE',
        'COL', 'DET', 'HOU', 'KCA', 'LAN', 'MIA', 'MIL', 'MIN', 'NYA',
        'NYN', 'OAK', 'PHI', 'PIT', 'SDN', 'SEA', 'SFN', 'SLN', 'TBA',
        'TEX', 'TOR', 'WAS']], dtype=object)

In [20]:
data = pyb.batting_stats(2024, qual=0)
data[data['Team'] == 'LAA']

,IDfg,Season,Name,Team,Age,G,AB,PA,H,1B,2B,3B,HR,R,RBI,BB,IBB,SO,HBP,SF,SH,GDP,SB,CS,AVG,GB,FB,LD,IFFB,Pitches,Balls,Strikes,IFH,BU,BUH,BB%,K%,BB/K,OBP,SLG,OPS,ISO,BABIP,GB/FB,LD%,GB%,FB%,IFFB%,HR/FB,IFH%,...,wCS/C (pi),wCU/C (pi),wFA/C (pi),wFC/C (pi),wFS/C (pi),wKN/C (pi),wSB/C (pi),wSI/C (pi),wSL/C (pi),wXX/C (pi),O-Swing% (pi),Z-Swing% (pi),Swing% (pi),O-Contact% (pi),Z-Contact% (pi),Contact% (pi),Zone% (pi),Pace (pi),FRM,AVG+,BB%+,K%+,OBP+,SLG+,ISO+,BABIP+,LD+%,GB%+,FB%+,HR/FB%+,Pull%+,Cent%+,Oppo%+,Soft%+,Med%+,Hard%+,EV,LA,Barrels,Barrel%,maxEV,HardHit,HardHit%,Events,CStr%,CSW%,xBA,xSLG,xwOBA,L-WAR
94,19858,2024,Luis Rengifo,LAA,27,61,219,237,68,53,10,1,4,30,20,13,0,33,4,0,1,3,20,6,0.311,106,42,38,6,850,283,567,6,1,0,0.055,0.139,0.39,0.360,0.420,0.780,0.110,0.352,0.0252,0.204,0.570,0.226,0.143,0.095,0.057,...,NaN,1.29,2.20,-0.61,0.97,-5.73,NaN,2.15,-0.50,NaN,0.354,0.757,0.552,0.647,0.870,0.797,0.491,18.1,NaN,130.0,67.0,63.0,117.0,107.0,71.0,124.0,1.05,136.0,58.0,85.0,100.0,106.0,90.0,109.0,102.0,93.0,86.5,5.2,3.0,0.016,110.2,60.0,0.321,187,0.112,0.224,0.259,0.340,0.290,1.2
133,17548,2024,Taylor Ward,LAA,30,75,274,316,66,40,13,0,13,37,40,36,0,74,1,5,0,6,3,3,0.241,68,97,40,4,1254,482,772,2,0,0,0.114,0.234,0.49,0.326,0.431,0.757,0.190,0.276,0.0070,0.195,0.332,0.473,0.041,0.134,0.029,...,NaN,-0.05,1.03,-1.51,-3.15,-6.70,NaN,-0.02,0.13,NaN,0.173,0.614,0.413,0.535,0.840,0.781,0.543,17.6,NaN,101.0,139.0,105.0,106.0,109.0,123.0,97.0,1.00,79.0,122.0,120.0,75.0,104.0,136.0,42.0,96.0,137.0,91.2,16.4,33.0,0.161,108.6,91.0,0.444,205,0.203,0.292,0.266,0.523,0.377,1.1
117,24729,2024,Logan O'Hoppe,LAA,24,66,231,250,62,39,12,1,10,34,33,14,0,62,3,1,1,5,1,0,0.268,70,59,41,6,962,330,632,5,1,0,0.056,0.248,0.23,0.317,0.459,0.776,0.190,0.325,0.0119,0.241,0.412,0.347,0.102,0.169,0.071,...,NaN,3.16,-0.40,-1.06,0.49,2.49,NaN,0.60,1.43,NaN,0.335,0.677,0.499,0.473,0.843,0.714,0.481,17.9,-2.7,112.0,68.0,112.0,103.0,117.0,123.0,115.0,1.24,98.0,90.0,151.0,128.0,80.0,82.0,90.0,83.0,136.0,90.5,13.7,18.0,0.105,111.5,82.0,0.480,171,0.156,0.297,0.271,0.481,0.343,1.5
49,10155,2024,Mike Trout,LAA,32,29,109,126,24,11,1,2,10,17,14,16,1,27,1,0,0,0,6,1,0.220,20,46,16,4,574,234,340,2,0,0,0.127,0.214,0.59,0.325,0.541,0.867,0.321,0.194,0.0043,0.195,0.244,0.561,0.087,0.217,0.100,...,NaN,2.26,1.49,-0.65,4.81,NaN,NaN,1.22,0.99,NaN,0.213,0.598,0.425,0.636,0.884,0.828,0.551,17.5,NaN,92.0,155.0,96.0,105.0,138.0,208.0,69.0,1.00,58.0,145.0,194.0,96.0,76.0,140.0,128.0,87.0,107.0,89.2,25.2,14.0,0.171,114.5,34.0,0.415,82,0.167,0.240,0.273,0.599,0.406,1.1
163,31347,2024,Zach Neto,LAA,23,75,256,276,65,38,17,0,10,29,32,13,0,67,5,1,1,4,9,6,0.254,82,68,36,9,1027,333,694,4,5,1,0.047,0.243,0.19,0.302,0.438,0.739,0.184,0.306,0.0121,0.194,0.441,0.366,0.132,0.147,0.049,...,NaN,-0.30,1.50,-0.97,-3.87,4.05,NaN,-1.57,0.88,NaN,0.333,0.695,0.521,0.476,0.838,0.727,0.520,18.3,NaN,106.0,58.0,109.0,98.0,111.0,119.0,108.0,0.99,105.0,95.0,131.0,121.0,92.0,75.0,68.0,103.0,111.0,89.7,13.2,18.0,0.094,111.3,73.0,0.382,191,0.149,0.289,0.244,0.449,0.321,0.9
248,19318,2024,Matt Thaiss,LAA,29,23,60,69,13,7,5,0,1,6,6,9,0,22,0,0,0,0,0,0,0.217,16,15,7,4,296,123,173,0,0,0,0.130,0.319,0.41,0.319,0.350,0.669,0.133,0.324,0.0107,0.184,0.421,0.395,0.267,0.067,0.000,...,NaN,1.64,1.04,-5.81,-4.35,NaN,NaN,2.85,-1.08,NaN,0.211,0.604,0.416,0.433,0.710,0.642,0.520,19.3,0.6,91.0,160.0,144.0,103.0,89.0,86.0,114.0,0.95,101.0,102.0,59.0,104.0,90.0,108.0,178.0,89.0,77.0,85.2,14.1,5.0,0.132,109.7,16.0,0.421,38,0.162,0.307,0.203,0.401,0.311,0.4
171,12164,2024,Miguel Sano,LAA,31,21,61,71,16,12,3,0,1,8,5,8,0,27,1,1,0,2,0,0,0.262,16,9,10,1,297,116,181,1,0,0,0.113,0.380,0.30,0.352,0.361,0.713,0.098,0.441,0.0178,0.286,0.457,0.257,0.111,0.111,0.063,...,NaN,-3.10,0.38,7.71,0.17,NaN,NaN,1.27,-3.23,NaN,0.283,0.632,0.453,0.302,0.791,0.634,0.486,18.1,NaN,110.0,138.0,171.0,114.0,92.0,64.0,155.0,1.47,109.0,66.0,99.0,120.0,73.0,106.0,53.0,59.0,196.0,97.0,12.1,4.0,0.114,114.9,22.0,0.629,35,0.155,0.320,0.260,0.400,0.330,0.2
385,8418,2024,E

In [74]:
pyb.playerid_lookup('pillar', 'kevin')

,name_last,name_first,key_mlbam,key_retro,key_bbref,key_fangraphs,mlb_played_first,mlb_played_last
0,pillar,kevin,607680,pillk001,pillake01,12434,2013.0,2024.0


In [55]:
ward = pyb.statcast_batter('2024-05-01', '2024-06-15', 621493)
ward.head(10)

,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,spin_dir,spin_rate_deprecated,break_angle_deprecated,break_length_deprecated,zone,des,game_type,stand,p_throws,home_team,away_team,type,hit_location,bb_type,balls,strikes,game_year,pfx_x,pfx_z,plate_x,plate_z,on_3b,on_2b,on_1b,outs_when_up,inning,inning_topbot,hc_x,hc_y,tfs_deprecated,tfs_zulu_deprecated,fielder_2,umpire,sv_id,vx0,vy0,vz0,ax,ay,az,sz_top,sz_bot,hit_distance_sc,launch_speed,launch_angle,effective_speed,release_spin_rate,release_extension,game_pk,pitcher.1,fielder_2.1,fielder_3,fielder_4,fielder_5,fielder_6,fielder_7,fielder_8,fielder_9,release_pos_y,estimated_ba_using_speedangle,estimated_woba_using_speedangle,woba_value,woba_denom,babip_value,iso_value,launch_speed_angle,at_bat_number,pitch_number,pitch_name,home_score,away_score,bat_score,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment,spin_axis,delta_home_win_exp,delta_run_exp,bat_speed,swing_length
0,SL,2024-06-14,83.9,-1.64,5.57,"Ward, Taylor",621493,678495,field_out,hit_into_play,NaN,NaN,NaN,NaN,9,Taylor Ward flies out sharply to center fielde...,R,R,R,SF,LAA,X,8.0,fly_ball,1,2,2024,0.91,-0.54,0.37,2.03,NaN,NaN,NaN,2,5,Top,153.73,46.44,NaN,NaN,672275,NaN,NaN,2.891235,-122.199726,-0.737368,8.552440,23.680042,-37.661803,3.39,1.61,383.0,101.2,25.0,85.0,2745,6.9,745326,678495,672275,527038,642731,656305,605113,624424,671218,596103,53.57,0.614,1.021000,0.0,1.0,0.0,0.0,6.0,48,4,Slider,1,8,8,1,8,1,8,1,Strategic,Standard,68,0.001,-0.065,72.193740,7.20065
1,SL,2024-06-14,83.4,-1.70,5.64,"Ward, Taylor",621493,678495,None,called_strike,NaN,NaN,NaN,NaN,11,Taylor Ward flies out sharply to center fielde...,R,R,R,SF,LAA,S,NaN,None,1,1,2024,1.09,-0.26,-0.85,2.78,NaN,NaN,NaN,2,5,Top,NaN,NaN,NaN,NaN,672275,NaN,NaN,-0.184304,-121.457337,0.350465,10.809557,23.502880,-35.011221,3.39,1.61,NaN,NaN,NaN,83.9,2684,6.6,745326,678495,672275,527038,642731,656305,605113,624424,671218,596103,53.93,NaN,NaN,NaN,NaN,NaN,NaN,NaN,48,3,Slider,1,8,8,1,8,1,8,1,Strategic,Standard,68,0.000,-0.024,NaN,NaN
2,SL,2024-06-14,84.8,-1.82,5.57,"Ward, Taylor",621493,678495,None,called_strike,NaN,NaN,NaN,NaN,6,Taylor Ward flies out sharply to center fielde...,R,R,R,SF,LAA,S,NaN,None,1,0,2024,0.65,0.04,0.72,2.30,NaN,NaN,NaN,2,5,Top,NaN,NaN,NaN,NaN,672275,NaN,NaN,4.653622,-123.481666,-1.422453,5.835143,23.495901,-31.725248,3.39,1.61,NaN,NaN,NaN,85.6,2767,6.7,745326,678495,672275,527038,642731,656305,605113,624424,671218,596103,53.84,NaN,NaN,NaN,NaN,NaN,NaN,NaN,48,2,Slider,1,8,8,1,8,1,8,1,Strategic,Standard,76,0.000,-0.018,NaN,NaN
3,FF,2024-06-14,98.7,-1.52,5.53,"Ward, Taylor",621493,678495,None,ball,NaN,NaN,NaN,NaN,8,Taylor Ward flies out sharply to center fielde...,R,R,R,SF,LAA,B,NaN,None,0,0,2024,-0.58,1.52,-0.08,1.50,NaN,NaN,NaN,2,5,Top,NaN,NaN,NaN,NaN,672275,NaN,NaN,5.304453,-143.407533,-9.100790,-9.200866,33.575619,-9.329478,3.39,1.61,NaN,NaN,NaN,99.4,2569,6.8,745326,678495,672275,527038,642731,656305,605113,624424,671218,596103,53.69,NaN,NaN,NaN,NaN,NaN,NaN,NaN,48,1,4-Seam Fastball,1,8,8,1,8,1,8,1,Strategic,Standard,203,0.000,0.014,NaN,NaN
4,FF,2024-06-14,94.4,-2.37,6.30,"Ward, Taylor",621493,675921,walk,ball,NaN,NaN,NaN,NaN,14,Taylor Ward walks.,R,R,R,SF,LAA,B,NaN,None,3,2,2024,-0.30,1.37,0.86,2.32,NaN,NaN,NaN,0,3,Top,NaN,NaN,NaN,NaN,672275,NaN,NaN,9.107832,-136.969575,-7.703605,-5.817816,30.234873,-13.368127,3.39,1.61,NaN,NaN,NaN,94.6,2093,6.6,745326,675921,672275,527038,642731,656305,605113,624424,671218,596103,53.91,NaN,0.692431,0.7,1.0,0.0,0.0,NaN,22,7,4-Seam Fastball,0,1,1,0,1,0,1,0,Strategic,Standard,208,-0.034,0.339,NaN,NaN
5,CU,2024-06-14,75.2,-2.08,6.37,"Ward, Taylor",621493,675921,None,ball,NaN,NaN,NaN,NaN,11,Taylor Ward walks.,R,R,R,SF,LAA,B,NaN,None,2,2,2024,0.45,-1.22,-1.87,3.13,NaN,NaN,NaN,0,3,Top,NaN,NaN,NaN,NaN,672275,NaN,NaN,-0.361987,-109.432048,2.623916,3.677476,20.090186,-42.640382,3.39,1.61,NaN,NaN,NaN,75